# 03 — EDA: Category Landscape

**Objective:** Map the competitive landscape of European food categories.

## Key Analyses
1. Brand concentration by category (HHI)
2. Private label penetration by category (per retailer)
3. Category size and assortment depth
4. Geographic variation in PL presence
5. Price tier analysis: median brand vs. median PL price per category

In [ ]:
from src.analysis.category_landscape import compute_hhi, compute_pl_penetration
from src.analysis.price_gaps import compute_price_gaps
from src.visualisation.category_map import plot_category_treemap, plot_hhi_bar

In [ ]:
# TODO: Load cleaned data and run landscape analysis